非常感谢您学习到这里，如果有时间的话希望能帮我填一个：[教程反馈](https://www.wjx.cn/jq/61058547.aspx)

在这个教程中，我想用taxi gps数据计算每一辆出租车在一天内的收入。如果能够成功计算，再看看高收入的出租车司机的行为，就可以分析出来：  
>毕业以后要是失业了，怎么样开出租车才能赚更多的钱！升职加薪赢取白富美就在明天！

提供的基础数据是：
<div class="alert alert-info"><h2>提供的基础数据是：</h2><p>   
    数据：<br>  
    1.出租车原始GPS数据<br>  

 </p></div>
 
 <div class="alert alert-info"><h2>深圳出租车计价规则：</h2><p>   
(一)起步价：首2公里11.00元;<br>  
(二)里程价：超过2公里部分，每公里2.40元;<br>  
(三)返空费：每天的23时至次日凌晨6时，超过25公里部分，每公里按上述里程价的30%加收返空费：<br>  
(四)夜间附加费：夜间起步价16元，每天的23时至次日凌晨6时，按上述起步价和里程价的20%加收夜间附加费;<br>  
(五)候时费：每分钟0.80元;<br>  
(六)大件行李费;体积超过0.2立方米、重量超过20公斤的大件行李，每件0.50元。<br>  
 </p></div>  
 
 
在计算的时候，我们可以以订单开始的时间判定夜间还是日间。另外，规则（六）也考虑不了

# 思路
## 首先，我们的输入和输出是什么？

输入很简单，TAXIGPS数据  
输出的形式，应该是：每条订单，收入  
为了后续的分析，在输出的基础上，再增加：订单号，收入，车牌号，订单开始时间，订单结束时间，订单上车坐标，订单下车坐标，行驶里程


那么，从输入怎么变成输出？计算出租车费，我们需要几个东西：订单号，行驶里程，订单开始时间（判断是否为夜间），候时的时长

## 任务分解

搞清楚输入和输出后，应用我们结构化数据的存储及处理的思维方式，就可以进行任务分解了，我们要解决的就是以下几个任务:  

    任务1：由最原始的输入，数据清洗，输出表1，字段与原始数据一致  
    
    任务2：表1输入，整理出每个订单的轨迹，用来计算里程和候时时长，输出表2，字段为：订单号，轨迹点经纬度，轨迹点的时间（订单号这一列需要我们自己定义）,速度（用来计算候时时长）  
    
    任务3：表2输入，计算每个订单的里程和候时时长，输出表3，字段为：订单号，订单的里程，候时时长  
    
    任务4：表1输入，整理出每个订单的其他信息，输出表4，字段为：订单号，车牌号，订单开始时间，订单结束时间，订单上车坐标，订单下车坐标  
    
    任务5：表3与表4输入，连接两个表，再计算订单收入，输出表5，字段为：订单号，收入，车牌号，订单开始时间，订单结束时间，订单上车坐标，订单下车坐标，行驶里程

# 数据

In [2]:
import pandas as pd

In [105]:

#读取数据
data2 = pd.read_csv(r'data-sample/TaxiData-Sample',header = None)
#给数据命名列
data2.columns = ['VehicleNum', 'Stime', 'Lng', 'Lat', 'OpenStatus', 'Speed']
data2.head(5)

,VehicleNum,Stime,Lng,Lat,OpenStatus,Speed
0,22271,22:54:04,114.167000,22.718399,0,0
1,22271,18:26:26,114.190598,22.647800,0,4
2,22271,18:35:18,114.201401,22.649700,0,0
3,22271,16:02:46,114.233498,22.725901,0,24
4,22271,21:41:17,114.233597,22.720900,0,19


# 订单收入计算

## 数据清洗

In [106]:
data2=data2.sort_values(by=['VehicleNum','Stime'])
#第一节课教的
data2 = data2[-((data2['OpenStatus'].shift(1)== data2['OpenStatus'].shift(-1))&\
      (data2['OpenStatus'].shift(1)!= data2['OpenStatus'])&\
      (data2['VehicleNum'].shift(1)==data2['VehicleNum'].shift(-1))&\
      (data2['VehicleNum']==data2['VehicleNum'].shift(1)))]
len(data2)

1598866

In [107]:
#将时间字符串转换为pd的时间格式，后面可以轻松的做加减
data2['Stime'] = pd.to_datetime(data2['Stime'])

## 订单号生成

如果StatusChange为这一条数据的OpenStatus减去上一条数据的OpenStatus，这样就会出现：

|OpenStatus     |   OpenStatus1    |  StatusChange||
| ----------- |-----------|||
|0          |       0    |             0||
|0          |       0    |             0||
|0         |        0    |             0||
|0          |       0    |             0 |    ←此时乘客上车了|
|1          |       0    |             1  |   ←此时乘客上车了|
|1          |       1    |             0||
|1          |       1    |             0||
|1          |       1    |             0||
|1          |       1    |             0||
|1          |       1    |             0||
|1          |       1    |             0||
|1          |       1    |             0||
|1          |       1    |             0||
|1          |       1    |             0|    ←此时乘客下车了|
|0          |       1    |             -1  |  ←此时乘客下车了|
|0          |       0    |             0||
|0          |       0    |             0||
|0          |       0    |             0||


那么如果我只把所有OpenStatus==1的数据拿出来，就会有：


|OpenStatus     |   orderid(订单号：对StatusChange的累加，可以用pandas的.cumsum()功能)    |  StatusChange||
| ----------- |-----------|||
|1          |       0    |             0||
|1          |       0    |             0||
|1         |        0    |             0||
|1          |       0    |             0 ||
|1          |       1    |             1  |   ←新订单|
|1          |       1    |             0||
|1          |       1    |             0||
|1          |       1    |             0||
|1          |       1    |             0||
|1          |       1    |             0||
|1          |       1    |             0||
|1          |       1    |             0||
|1          |       1    |             0||
|1          |       1    |             0||
|1          |       2   |             1  |  ←新订单|
|1          |       2    |             0||
|1          |       2    |             0||
|1          |       2    |             0||


In [109]:
data2['OpenStatus1'] = data2['OpenStatus'].shift()
data2['VehicleNum1'] = data2['VehicleNum'].shift()

data2['Lng1'] = data2['Lng'].shift()
data2['Lat1'] = data2['Lat'].shift()
data2['Stime1'] = data2['Stime'].shift()

data2['StatusChange'] = data2['OpenStatus1']-data2['OpenStatus']


data2 = data2[data2['OpenStatus1']==1]
data2['orderid'] = data2['StatusChange'].cumsum()
data2.head(30)

,VehicleNum,Stime,Lng,Lat,OpenStatus,Speed,OpenStatus1,VehicleNum1,Lng1,Lat1,Stime1,StatusChange,orderid
1550370,22334,2020-01-23 00:01:04,114.111130,22.576750,0,0,1.0,22334.0,114.111130,22.576750,2020-01-23 00:00:52,1.0,1.0
1549617,22334,2020-01-23 00:08:37,114.080551,22.554251,1,0,1.0,22334.0,114.080551,22.554251,2020-01-23 00:08:17,0.0,1.0
1547008,22334,2020-01-23 00:08:57,114.080551,22.554251,1,0,1.0,22334.0,114.080551,22.554251,2020-01-23 00:08:37,0.0,1.0
1548874,22334,2020-01-23 00:09:17,114.081131,22.552799,1,15,1.0,22334.0,114.080551,22.554251,2020-01-23 00:08:57,0.0,1.0
1550335,22334,2020-01-23 00:09:37,114.080681,22.551718,1,30,1.0,22334.0,114.081131,22.552799,2020-01-23 00:09:17,0.0,1.0
1547678,22334,2020-01-23 00:09:57,114.078049,22.551600,1,50,1.0,22334.0,114.080681,22.551718,2020-01-23 00:09:37,0.0,1.0
1549818,22334,2020-01-23 00:10:17,114.076401,22.551550,1,30,1.0,22334.0,114.078049,22.551600,2020-01-23 00:09:57,0.0,1.0
1548021,22334,2020-01-23 00:10:37,114.076401,22.551550,1,0,1.0,22334.0,114.076401,22.551550,2020-01-23 00:10:17,0.0,1.0
1548022,22334,2020-01-23 00:10:57,114.076401,22.551550,1,0,1.0,22334.0,114.076401,22.551550,2020-01-23 00:10:37,0.0,1.0
1548178,22334,2020-01-23 00:11:17,114.076401,22.551550,1,0,1.0,22334.0,114.076401,22.551550,2020-01-23 00:10:57,0.0,1.0


In [102]:
#表2入手
table2 = data2[['orderid','Lng','Lat','Stime','Speed']]
table2.head(5)

,orderid,Lng,Lat,Stime,Speed
1548741,0.0,114.111130,22.576750,2020-01-23 00:00:52,13
1550370,1.0,114.111130,22.576750,2020-01-23 00:01:04,0
1549616,1.0,114.080551,22.554251,2020-01-23 00:08:17,0
1549617,1.0,114.080551,22.554251,2020-01-23 00:08:37,0
1547008,1.0,114.080551,22.554251,2020-01-23 00:08:57,0


## 订单里程计算与候车时间

In [5]:
#表2
#计算里程

#定义计算路径长度函数

from math import pi
import numpy as np

def getdistance(lon1, lat1, lon2, lat2): # 经度1，纬度1，经度2，纬度2 （十进制度数）输入为DataFrame的列
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # 将十进制度数转化为弧度
    lon1, lat1, lon2, lat2 = map(lambda r:r*pi/180, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(a**0.5) 
    r = 6371 # 地球平均半径，单位为公里
    return c * r * 1000




In [41]:
#生成订单的里程

table3 = table2.copy()
table3['Lng1'] = table3['Lng'].shift(-1)
table3['Lat1'] = table3['Lat'].shift(-1)
table3['orderid1'] = table3['orderid'].shift(-1)
table3['Stime1'] = table3['Stime'].shift(-1)

table3 = table3[(table3['orderid1'] == table3['orderid'])]
#计算每个点与下一个点的距离
lon1 = table3['Lng']
lat1 = table3['Lat']
lon2 = table3['Lng1']
lat2 = table3['Lat1']
table3['distance'] = getdistance(lon1, lat1, lon2, lat2)


#计算每个点与下一个点的时间差
table3['interval'] = (table3['Stime1']-table3['Stime']).apply(lambda r:r.seconds)

#集计得到出租车路径长度
orderlenth = table3[['orderid','distance']].groupby('orderid').sum().reset_index()

waittime = table3[table3['distance']==0][['orderid','interval']].groupby('orderid').sum()


table3 = pd.merge(orderlenth,waittime,on = 'orderid',how='left')
table3.head(10)

,orderid,distance,interval
0,1.0,6785.602632,180.0
1,2.0,15698.063306,309.0
2,3.0,12508.941904,140.0
3,4.0,15144.760499,100.0
4,5.0,15458.541238,60.0
5,6.0,3797.074876,80.0
6,7.0,7497.547127,40.0
7,8.0,4287.367646,60.0
8,9.0,5056.517539,300.0
9,10.0,2732.022522,200.0


## 生成表4

In [75]:
#表4的字段
#订单号，车牌号，订单开始时间，订单结束时间，订单上车坐标，订单下车坐标  

In [45]:
#将每一个出行的起点与终点提取出来
o = data2.iloc[:1].append(data2[data2['StatusChange']==1])
d = data2[(data2['StatusChange']==1).shift(-1).fillna(False)]
table4 = o.append(d).sort_values(by = ['orderid','Stime'])[['orderid','VehicleNum','Stime','Lng','Lat']]
table4.head(5)

,orderid,VehicleNum,Stime,Lng,Lat
1548741,0.0,22334,2020-01-20 00:00:52,114.111130,22.57675
1548741,0.0,22334,2020-01-20 00:00:52,114.111130,22.57675
1550370,1.0,22334,2020-01-20 00:01:04,114.111130,22.57675
1549620,1.0,22334,2020-01-20 00:17:58,114.084915,22.54085
1550727,2.0,22334,2020-01-20 00:18:16,114.084915,22.54085


In [46]:
#加一列isd，如果该行为起点，则isd=0，如果该行为终点，则isd=1
table4['isd'] = [i%2 for i in range(len(table4))]
table4.head(5)

,orderid,VehicleNum,Stime,Lng,Lat,isd
1548741,0.0,22334,2020-01-20 00:00:52,114.111130,22.57675,0
1548741,0.0,22334,2020-01-20 00:00:52,114.111130,22.57675,1
1550370,1.0,22334,2020-01-20 00:01:04,114.111130,22.57675,0
1549620,1.0,22334,2020-01-20 00:17:58,114.084915,22.54085,1
1550727,2.0,22334,2020-01-20 00:18:16,114.084915,22.54085,0


In [49]:
# 把O与D的信息放在同一行
table4['Etime'] = table4['Stime'].shift(-1)
table4['ELng'] = table4['Lng'].shift(-1)
table4['ELat'] = table4['Lat'].shift(-1)
table4 = table4[(table4['isd']==0)&(-table4['Etime'].isnull())]

In [54]:
#连接表3与表4成为表5
table5 = pd.merge(table4,table3,on = 'orderid',how = 'left')
table5.head(5)

,orderid,VehicleNum,Stime,Lng,Lat,isd,Etime,ELng,ELat,distance,interval
0,0.0,22334,2020-01-20 00:00:52,114.111130,22.576750,0,2020-01-20 00:00:52,114.111130,22.576750,NaN,NaN
1,1.0,22334,2020-01-20 00:01:04,114.111130,22.576750,0,2020-01-20 00:17:58,114.084915,22.540850,6785.602632,180.0
2,2.0,22334,2020-01-20 00:18:16,114.084915,22.540850,0,2020-01-20 00:44:47,114.056236,22.633383,15698.063306,309.0
3,3.0,22334,2020-01-20 00:44:52,114.056236,22.633383,0,2020-01-20 02:46:52,114.093498,22.554382,12508.941904,140.0
4,4.0,22334,2020-01-20 02:47:04,114.093536,22.554382,0,2020-01-20 04:13:57,114.052299,22.604366,15144.760499,100.0


In [57]:
#每天的23时至次日凌晨6时为夜间
table5['isnight'] = (table5['Stime'].apply(lambda r:r.hour)<6)|(table5['Stime'].apply(lambda r:r.hour)>=23)
table5.head(5)

,orderid,VehicleNum,Stime,Lng,Lat,isd,Etime,ELng,ELat,distance,interval,isnight
0,0.0,22334,2020-01-20 00:00:52,114.111130,22.576750,0,2020-01-20 00:00:52,114.111130,22.576750,NaN,NaN,True
1,1.0,22334,2020-01-20 00:01:04,114.111130,22.576750,0,2020-01-20 00:17:58,114.084915,22.540850,6785.602632,180.0,True
2,2.0,22334,2020-01-20 00:18:16,114.084915,22.540850,0,2020-01-20 00:44:47,114.056236,22.633383,15698.063306,309.0,True
3,3.0,22334,2020-01-20 00:44:52,114.056236,22.633383,0,2020-01-20 02:46:52,114.093498,22.554382,12508.941904,140.0,True
4,4.0,22334,2020-01-20 02:47:04,114.093536,22.554382,0,2020-01-20 04:13:57,114.052299,22.604366,15144.760499,100.0,True


## 收入计算

(一)起步价：首2公里11.00元;  
(二)里程价：超过2公里部分，每公里2.40元;  
(三)返空费：每天的23时至次日凌晨6时，超过25公里部分，每公里按上述里程价的30%加收返空费：  
(四)夜间附加费：夜间起步价16元，每天的23时至次日凌晨6时，按上述起步价和里程价的20%加收夜间附加费;  
(五)候时费：每分钟0.80元;  

In [63]:
#起步价
table5['起步价'] = table5['isnight']*(16-11)+11

In [64]:
#里程价
table5['里程价'] = ((table5['distance']-2000)>0)*(table5['distance']-2000)*2.4/1000

In [65]:
#返空费
table5['返空费'] = table5['isnight']*((table5['distance']-25000)>0)*((table5['distance']-25000)*2.4*0.3/1000)

In [66]:
#夜间附加费(里程价)
table5['夜间附加费'] = table5['isnight']*((table5['distance']-2000)>0)*(table5['distance']-2000)*2.4*0.2/1000

In [67]:
#候时费
table5['候时费'] = table5['interval']/60*0.8

In [68]:
#最终价格
table5['price'] = table5['起步价'] + table5['里程价'] + table5['返空费'] + table5['夜间附加费'] + table5['候时费']
table5.head(5)

,orderid,VehicleNum,Stime,Lng,Lat,isd,Etime,ELng,ELat,distance,interval,isnight,起步价,里程价,返空费,夜间附加费,候时费,price
0,0.0,22334,2020-01-20 00:00:52,114.111130,22.576750,0,2020-01-20 00:00:52,114.111130,22.576750,NaN,NaN,True,16,NaN,NaN,NaN,NaN,NaN
1,1.0,22334,2020-01-20 00:01:04,114.111130,22.576750,0,2020-01-20 00:17:58,114.084915,22.540850,6785.602632,180.0,True,16,11.485446,-0.0,2.297089,2.400000,32.182536
2,2.0,22334,2020-01-20 00:18:16,114.084915,22.540850,0,2020-01-20 00:44:47,114.056236,22.633383,15698.063306,309.0,True,16,32.875352,-0.0,6.575070,4.120000,59.570422
3,3.0,22334,2020-01-20 00:44:52,114.056236,22.633383,0,2020-01-20 02:46:52,114.093498,22.554382,12508.941904,140.0,True,16,25.221461,-0.0,5.044292,1.866667,48.132419
4,4.0,22334,2020-01-20 02:47:04,114.093536,22.554382,0,2020-01-20 04:13:57,114.052299,22.604366,15144.760499,100.0,True,16,31.547425,-0.0,6.309485,1.333333,55.190244


In [69]:
#筛选掉没有计算出来价格的数据
table5 = table5[-table5['price'].isnull()]
table5.head(5)

,orderid,VehicleNum,Stime,Lng,Lat,isd,Etime,ELng,ELat,distance,interval,isnight,起步价,里程价,返空费,夜间附加费,候时费,price
1,1.0,22334,2020-01-20 00:01:04,114.111130,22.576750,0,2020-01-20 00:17:58,114.084915,22.540850,6785.602632,180.0,True,16,11.485446,-0.0,2.297089,2.400000,32.182536
2,2.0,22334,2020-01-20 00:18:16,114.084915,22.540850,0,2020-01-20 00:44:47,114.056236,22.633383,15698.063306,309.0,True,16,32.875352,-0.0,6.575070,4.120000,59.570422
3,3.0,22334,2020-01-20 00:44:52,114.056236,22.633383,0,2020-01-20 02:46:52,114.093498,22.554382,12508.941904,140.0,True,16,25.221461,-0.0,5.044292,1.866667,48.132419
4,4.0,22334,2020-01-20 02:47:04,114.093536,22.554382,0,2020-01-20 04:13:57,114.052299,22.604366,15144.760499,100.0,True,16,31.547425,-0.0,6.309485,1.333333,55.190244
5,5.0,22334,2020-01-20 04:23:07,114.052216,22.602118,0,2020-01-20 06:41:19,114.067886,22.521299,15458.541238,60.0,True,16,32.300499,-0.0,6.460100,0.800000,55.560599


In [70]:
#保存
table5.to_csv(r'data-sample/taxi-price.csv',index = None)